In [1]:
from Assemblies.game import GameField
import numpy as np
from numpy import random
from sys import exit as sys_exit

%matplotlib inline
import matplotlib.pyplot as plt

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, Layout, VBox
import ipywidgets as widgets

рассмтореть зверей, которые появились у меня

рассмотреть, что происходит с конкретными фигурами в разных режимах

In [2]:
b_points = np.array([1, 7/6, 6/5, 5/4, 4/3, 7/5, 3/2, 5/3, 7/4, 2, 7/3, 5/2]) + 0.01

In [3]:
def show_field(field, point_size=10, scale=10):
    plt.clf()
    fig = plt.figure(figsize = (scale*1, scale*0.666))
    point_size = scale / (len(field)**2) * 10000
    
    
    y, x = (1-field).nonzero()
    plt.scatter(x + y*np.sin(np.pi/6), y * np.sin(np.pi/3), s=point_size, marker='h')
    y, x = field.nonzero()
    plt.scatter(x + y*np.sin(np.pi/6), y * np.sin(np.pi/3), s=point_size, marker='h', c='r')
    plt.show()

In [4]:
def get_field(b):
    show_field(fields[b].field, scale=20)

In [5]:
#animal - массив номеров элементов зверя (количество строк - 2, количество столбцов = количество элементов)
#по умолчанию зверь дефекторный
def init(size, animal, defect=1):
    fields = {b : GameField(size, b) for b in b_points}
    
    for b in b_points:
        if defect==1:
            fields[b].field = np.zeros((size, size), dtype=int)

            if animal.shape[0]!=2:
                sys_exit('неверная размерность матрицы')

            quantity=len(animal[0])

            if quantity>size**2:
                sys_exit('количество элементов в звере превышает количество элементов на решётке')

            if np.max(animal)>size-1:
                sys_exit('горизонтальный и/или вертикальный размер зверя превышает размер решётки')

            for i in range(quantity):
                fields[b].field[animal[1][i]][animal[0][i]] = 1
        else:
            fields[b].field = np.ones((size, size), dtype=int)

            if animal.shape[0]!=2:
                sys_exit('неверная размерность матрицы')

            quantity=len(animal[0])

            if quantity>size**2:
                sys_exit('количество элементов в звере превышает количество элементов на решётке')

            if np.max(animal)>size-1:
                sys_exit('горизонтальный и/или вертикальный размер зверя превышает размер решётки')

            for i in range(quantity):
                fields[b].field[animal[1][i]][animal[0][i]] = 0
        
    return fields

## Initialization

In [6]:
#Init
size_int = widgets.BoundedIntText(
    value=3,
    min=3,
    max=1000,
    step=3,
    description='Size:',
    disabled=False
)
init_btn = widgets.ToggleButton(
    value=False,
    description='Init',
    disabled=False,
    button_style=''
)
def on_init_btn(current):
    global fields
    fields = init(size_int.value,draw_animal,defect)
    
    
#Slider   
b_sldr = widgets.Dropdown(options=b_points, description='b:', continuous_update=False)


#Steps
step_count_int = widgets.BoundedIntText(
    value=1,
    min=1,
    max=1000,
    step=1,
    description='Count:',
    disabled=False
)

step_btn = widgets.ToggleButton(
    value=False,
    description='Step',
    disabled=False,
    button_style=''
)
def on_step_btn(current):
    current.value = False
    global step_count_int
    GameField.evolve(fields[b_sldr.value], num_steps=step_count_int.value)
        
def f(init, b, step):
    get_field(b)

In [7]:
size=33
fields = init(size,np.array([[0],[0]]))

def draw(animal,defector=1):    
    global draw_animal
    draw_animal=animal
    
    global defect
    defect=defector
    
    init_btn.observe(on_init_btn, names='value')
    step_btn.observe(on_step_btn, names='value')
    
    ui = widgets.HBox([size_int, init_btn, b_sldr, step_count_int, step_btn])
    out = widgets.interactive_output(f, {'init': init_btn, 'b': b_sldr, 'step' : step_btn})

    display(ui, out)

При 1<b<7/6

При 7/6<b<6/5

При 6/5<b<5/4

При 5/4<b<4/3

При 4/3<b<7/5

При 7/5<b<3/2

При 3/2<b<5/3

При 5/3<b<7/4

При 7/4<b<2

При 2<b<7/3

При 7/3<b<5/2

При 5/2<b<3

# дефекторный зверь

### 70%, 72*72, 5/4<b<4/3

### "омега"

In [8]:
omega=np.array([[0,1,2,3,4,5,6,5,5,5,4,5,8,5,9,6,9,7,8],[4,4,4,4,4,4,4,5,6,7,3,3,3,2,2,1,1,0,0]])+10

In [9]:
draw(six_rom,0)

HBox(children=(BoundedIntText(value=3, description='Size:', max=1000, min=3, step=3), ToggleButton(value=False, description='Init'), Dropdown(description='b:', options=(1.01, 1.1766666666666667, 1.21, 1.26, 1.3433333333333333, 1.41, 1.51, 1.6766666666666667, 1.76, 2.01, 2.3433333333333333, 2.51), value=1.01), BoundedIntText(value=1, description='Count:', max=1000, min=1), ToggleButton(value=False, description='Step')))

Output()

При 1<b<7/6 вырождается в 2 точки.

При 7/6<b<6/5 точки со сдвигом по фазе превращаются в ромашек.

При 6/5<b<5/4 есть две пары стабильных точек. Есть мигающая точка-ромашка.

При 5/4<b<4/3 омега сохраняется, мигание тройки дефекторов.

При 4/3<b<7/5 омега колеблется с периодом 4.

При 7/5<b<3/2 новый зверь колеблется с периодом 2 - гантеля с точкой и линия с ромашкой.

При 3/2<b<5/3 новый, другой зверь колеблется с периодом 2 - две точки вместе превращаются в ромашки.

При 5/3<b<7/4 новый, другой зверь колеблется с периодом 2 - мигание точки-ромашки.

При 7/4<b<2 новый, другой зверь колеблется с периодом 2 - перемещение ромашки.

При 2<b<7/3 новый, лругой зверь колеблется с периодом 2 - перемещается точка.

При 7/3<b<5/2 зверь разрастается до размеров поля, оставляя "ромашки" кооператоров. Если 2 ромашки расположены на расстоянии одного дефектора друг от друга, то все подобные пары колеблются в одной фазе с периодом 2.

При 5/2<b<3 зависит от размера. При 72x72 зверь разрастается на всё поле, оставляя 2 стабильных кооператорных треугольника с округлыми концами. В остальных проверенных случаях, в т.ч. при 69 и 75 всё поле дефекторное.

### тройка дефекторов

In [21]:
triplet=np.array([[0,1,0],[0,0,1]])+10

При 1<b<7/6 зверь пропадает

При 7/6<b<6/5 -//-

При 6/5<b<5/4 -//-

При 5/4<b<4/3 -//-

При 4/3<b<7/5 -//-

При 7/5<b<3/2 -//-

При 3/2<b<5/3 зверь не меняется, стабилен.

При 5/3<b<7/4 -//-

При 7/4<b<2 колебания зверей с полным периодом 3.

При 2<b<7/3 стабильный треугольник с округлыми концами.

При 7/3<b<5/2 зверь разрастается до размеров поля, оставляя "ромашки" кооператоров. Если 2 ромашки расположены на расстоянии одного дефектора друг от друга, то все подобные пары колеблются в одной фазе с периодом 2.

При 5/2<b<3 зависит от размера. До 144 остаются 1-2 кооператорных зверя, выше - сплошное дефекторное поле.

### две точки

In [10]:
two_dots=np.array([[1,0],[0,3]])+10

При 1<b<7/6 зверь стабилен.

При 7/6<b<6/5 превращение в ромашки с периодом 2.

При 6/5<b<5/4 превращение в ромашки с периодом 2.

При 5/4<b<4/3 превращение в ромашки с периодом 2.

При 4/3<b<7/5 превращение в ромашки, потом в змейку. Зверь стабилен.

При 7/5<b<3/2 чередование ромашки со змейкой с периодом 2.

При 3/2<b<5/3 чередование ромашки со змейкой с периодом 2.

При 5/3<b<7/4 превращение ромашки в параллелограмм. Зверь стабилен.

При 7/4<b<2 чередование ромашки с параллелограммом.

При 2<b<7/3 стабильные ромашки.

При 7/3<b<5/2 зверь разрастается до размеров поля, оставляя "ромашки" кооператоров. Если 2 ромашки расположены на расстоянии одного дефектора друг от друга, то все подобные пары колеблются в одной фазе с периодом 2.

При 5/2<b<3 зависит от размера. При 114x114 зверь разрастается на всё поле оставляя 2 стабильных ромашки. В остальных проверенных случаях, в т.ч. при 111 и 117 всё поле дефекторное. 

### "ромашка"

In [8]:
romashka=np.array([[1,2,0,1,2,0,1],[0,0,1,1,1,2,2]])+10

При 1<b<7/6 вырождается в точку.

При 7/6<b<6/5 чередование точки и ромашки.

При 6/5<b<5/4 -//-

При 5/4<b<4/3 -//-

При 4/3<b<7/5 -//-

При 7/5<b<3/2 -//-

При 3/2<b<5/3 -//-

При 5/3<b<7/4 -//-

При 7/4<b<2 -//-

При 2<b<7/3 стабильная ромашка.

При 7/3<b<5/2 зверь разрастается до размеров поля, оставляя "ромашки" кооператоров. Если 2 ромашки расположены на расстоянии одного дефектора друг от друга, то все подобные пары колеблются в одной фазе с периодом 2.

При 5/2<b<3 зависит от размера. При 69x69 и 72x72 зверь разрастается на всё поле, оставляя 2 стабильных кооператорных треугольника с округлыми концами. В остальных проверенных случаях, в т.ч. при 66 и 75 всё поле дефекторное.

# кооператорный зверь

### 70%, 72*72, 7/3<b<3

### "треугольник с округлыми концами"

In [8]:
triangle=np.array([[0,1,0,1,2,0,1,2,3,0,1,2,3,4,1,2,3,4],[4,4,3,3,3,2,2,2,2,1,1,1,1,1,0,0,0,0]])+10

При 1<b<7/6 зверь разрастается на всё поле.

При 7/6<b<6/5 -//-

При 6/5<b<5/4 -//-

При 5/4<b<4/3 зверь разрастается на всё поле оставляя "Y"-образную линию точек дефекторов, которые мигают с ромашками с периодом 2.

При 4/3<b<7/5 оставляет три точки сверху-справа, которые мигают с ромашками с периодом 2.

При 7/5<b<3/2 -//-

При 3/2<b<5/3 "Y"-образная стабильная линия четвёрок-дефекторов

При 5/3<b<7/4 -//-
 
При 7/4<b<2 ещё более плотная "Y"-образная линия дефекторов, полный период колебания - 3.

При 2<b<7/3 стал шестиконечной звездой и остался ей.

При 7/3<b<5/2 периодически превращается в разных зверей. Полный период - 6.

При 5/2<b<3 - зверь не меняется, стабилен.

### Шесть ромашек - результат одной ромашки при 7/3<b<5/2

In [8]:
#Слева против часовой
romashka1=np.array([[1,2,0,1,2,0,1],[0,0,1,1,1,2,2]])+10
romashka2=romashka1+np.array([[6],[-3]])
romashka3=romashka1+np.array([[9],[0]])
romashka4=romashka3+np.array([[-3],[6]])
romashka6=romashka1+np.array([[-3],[6]])
romashka5=romashka6+np.array([[3],[3]])

st1=np.hstack((romashka1,romashka2))
st2=np.hstack((st1,romashka3))
st3=np.hstack((st2,romashka4))
st4=np.hstack((st3,romashka5))
six_rom=np.hstack((st4,romashka6))

При 1<b<7/6 исчезает, оставляя "X"-образную прерывистую линию единичек-дефекторов.

При 7/6<b<6/5 единички этой линии мигают с ромашками со сдвигом по фазе (нижний дефектор отдельно от остальных). Полный период - 2. 

При 6/5<b<5/4 линии дефекторов (промежуток между дефекторами сузился) пересекаются в звезду, которая расположена в том самом нижнем дефекторе). В центре свободных участков чередование единичек дефекторов с ромашками.

При 5/4<b<4/3 линии звезды сплошные. Мигание центра звезды с "Y"-образной линией (по форме похожа на ту, что была при "треугольнике с округлыми концами") со свдигом по фазе. Период - 2.

При 4/3<b<7/5 центр звезды всё так же мигает с периодом 2. От "Y"-образной линии остались точки в узлах. Колеблются в противофазе со звездой.

При 7/5<b<3/2 -//-

При 3/2<b<5/3 снова вернулась "Y"-образная линия. Её центры колеблются со свдигом фазы с центром звезды.

При 5/3<b<7/4 -//-

При 7/4<b<2 звезда и Y колеблются с периодом 3.

При 2<b<7/3 начальный зверь стабилен.

При 7/3<b<5/2 -//-

При 5/2<b<3 -//-